# Results and DSS Operations

This notebook demonstrates DSS file operations and results extraction:

| Class | Purpose |
|-------|--------|
| `HmsDss` | Low-level DSS file operations (catalog, read, write) |
| `HmsResults` | High-level results extraction (peak flows, hydrographs, volumes) |

## Prerequisites

DSS operations require:
- Java 8+ JDK/JRE installed
- `pyjnius` package (`pip install pyjnius`)

The HEC Monolith libraries are auto-downloaded on first use.

In [ ]:
# pip install hms-commander

**For Development**: If working on hms-commander source code, use the `hmscmdr_local` conda environment (editable install) instead of pip install.

In [ ]:
from pathlib import Path
from hms_commander import (
    HmsExamples,
    init_hms_project,
    HmsCmdr,
    HmsDss,
    HmsResults
)

print("hms-commander loaded")

# Check if DSS operations are available
dss_available = HmsDss.is_available()
print(f"DSS operations available: {dss_available}")

if not dss_available:
    print("\nNote: DSS operations require Java and pyjnius.")
    print("Install with: pip install pyjnius")
    print("Ensure JAVA_HOME is set to a Java 8+ JDK.")

## 1. Extract Example Project and Generate Results

In [ ]:
# Extract the tifton example project
project_path = HmsExamples.extract_project(
    "tifton",
    output_path=Path.cwd() / 'hms_example_projects' / 'tifton_results_dss'
)

print(f"Project extracted to: {project_path}")

# Initialize project
hms = init_hms_project(project_path)
print(f"\nProject: {hms.project_name}")
print(f"Available runs: {hms.list_run_names()}")

In [ ]:
# Execute a run to generate DSS results (if needed)
run_name = hms.list_run_names()[0]
dss_file = hms.get_run_dss_file(run_name)

print(f"Run: {run_name}")
print(f"DSS file: {dss_file}")
print(f"DSS exists: {dss_file.exists() if dss_file else False}")

# Run HMS if DSS doesn't exist
if dss_file and not dss_file.exists():
    print(f"\nExecuting {run_name} to generate DSS results...")
    try:
        result = HmsCmdr.compute_run(run_name, hms_object=hms)
        if result.success:
            print(f"[OK] Completed in {result.execution_time:.1f}s")
        else:
            print(f"[FAILED] {result.error_message}")
    except Exception as e:
        print(f"[SKIP] HMS execution not available: {e}")

## 2. DSS File Information

Get metadata about the DSS file.

In [ ]:
if dss_available and dss_file and dss_file.exists():
    # Get DSS file info
    info = HmsDss.get_info(str(dss_file))
    
    print("DSS File Information:")
    print("=" * 60)
    for key, value in info.items():
        print(f"  {key}: {value}")
else:
    print("DSS file not available for analysis")

## 3. DSS Catalog Exploration

The catalog lists all pathnames (data records) in the DSS file.

In [ ]:
if dss_available and dss_file and dss_file.exists():
    # Get DSS catalog
    catalog = HmsDss.get_catalog(str(dss_file))
    
    print(f"DSS Catalog ({len(catalog)} pathnames):")
    print("=" * 60)
    
    # Show first 15 pathnames
    for path in catalog[:15]:
        print(f"  {path}")
    
    if len(catalog) > 15:
        print(f"  ... and {len(catalog) - 15} more")
else:
    print("DSS catalog not available")

### Filter Catalog by Data Type

In [ ]:
if dss_available and dss_file and dss_file.exists():
    # Filter for different data types
    flow_paths = [p for p in catalog if 'FLOW' in p.upper()]
    precip_paths = [p for p in catalog if 'PRECIP' in p.upper()]
    depth_paths = [p for p in catalog if 'DEPTH' in p.upper()]
    
    print("Pathnames by Data Type:")
    print("=" * 60)
    print(f"  Flow:          {len(flow_paths)} paths")
    print(f"  Precipitation: {len(precip_paths)} paths")
    print(f"  Depth/Stage:   {len(depth_paths)} paths")
    
    # Show flow paths
    if flow_paths:
        print("\nFlow Pathnames:")
        for path in flow_paths[:10]:
            print(f"  {path}")

## 4. Read Time Series Data

Read individual time series from the DSS file.

In [ ]:
if dss_available and dss_file and dss_file.exists() and flow_paths:
    # Read first flow time series
    pathname = flow_paths[0]
    print(f"Reading: {pathname}")
    print("=" * 60)
    
    df = HmsDss.read_timeseries(str(dss_file), pathname)
    
    print(f"\nTime Series Properties:")
    print(f"  Shape: {df.shape}")
    print(f"  Time range: {df.index.min()} to {df.index.max()}")
    print(f"  Units: {df.attrs.get('units', 'N/A')}")
    print(f"  Data type: {df.attrs.get('data_type', 'N/A')}")
    
    print(f"\nFirst 10 values:")
    display(df.head(10))
else:
    print("Time series reading not available")

## 5. Extract Peak Flows (HmsResults)

`HmsResults.get_peak_flows()` extracts peak discharge for all flow elements.

In [ ]:
if dss_available and dss_file and dss_file.exists():
    try:
        peaks = HmsResults.get_peak_flows(str(dss_file))
        
        print("Peak Flows:")
        print("=" * 60)
        display(peaks)
        
        # Validate: all peaks should be positive
        if not peaks.empty and 'peak_flow' in peaks.columns:
            all_positive = (peaks['peak_flow'] > 0).all()
            print(f"\n[{'OK' if all_positive else 'WARNING'}] All peak flows positive: {all_positive}")
    except Exception as e:
        print(f"Could not extract peak flows: {e}")
else:
    print("Peak flow extraction not available")

## 6. Extract Hydrograph Time Series (HmsResults)

`HmsResults.get_outflow_timeseries()` extracts the full hydrograph for a specific element.

In [ ]:
if dss_available and dss_file and dss_file.exists() and flow_paths:
    # Extract element name from first flow path
    # DSS path format: /A/B/C/D/E/F/ where B is typically the element name
    parts = flow_paths[0].split('/')
    element_name = parts[2] if len(parts) > 2 else None
    
    if element_name:
        print(f"Extracting hydrograph for: {element_name}")
        print("=" * 60)
        
        try:
            hydrograph = HmsResults.get_outflow_timeseries(
                str(dss_file),
                element_name
            )
            
            if hydrograph is not None and not hydrograph.empty:
                print(f"\nHydrograph Statistics:")
                print(f"  Duration: {hydrograph.index.max() - hydrograph.index.min()}")
                print(f"  Peak: {hydrograph['flow'].max():.2f} CFS")
                print(f"  Time to peak: {hydrograph['flow'].idxmax()}")
                print(f"  Mean flow: {hydrograph['flow'].mean():.2f}")
                
                display(hydrograph.head(10))
            else:
                print(f"No hydrograph data found for {element_name}")
        except Exception as e:
            print(f"Could not extract hydrograph: {e}")
else:
    print("Hydrograph extraction not available")

## 7. Visualize Hydrograph

In [ ]:
try:
    import matplotlib.pyplot as plt
    
    if 'hydrograph' in dir() and hydrograph is not None and not hydrograph.empty:
        fig, ax = plt.subplots(figsize=(12, 6))
        
        ax.plot(hydrograph.index, hydrograph['flow'], 'b-', linewidth=1.5)
        ax.fill_between(hydrograph.index, hydrograph['flow'], alpha=0.3)
        
        ax.set_xlabel('Time')
        ax.set_ylabel('Flow (CFS)')
        ax.set_title(f"Hydrograph - {element_name}")
        ax.grid(True, alpha=0.3)
        
        # Mark peak
        peak_idx = hydrograph['flow'].idxmax()
        peak_val = hydrograph['flow'].max()
        ax.axhline(y=peak_val, color='r', linestyle='--', alpha=0.5)
        ax.annotate(f'Peak: {peak_val:.1f}', 
                    xy=(peak_idx, peak_val),
                    xytext=(10, 10), textcoords='offset points',
                    fontsize=10, color='red')
        
        plt.tight_layout()
        plt.show()
    else:
        print("No hydrograph data available for plotting")
        
except ImportError:
    print("matplotlib not installed - visualization skipped")

## 8. Compare Multiple Elements

In [ ]:
if dss_available and dss_file and dss_file.exists():
    # Get unique element names from flow paths
    elements = set()
    for path in flow_paths:
        parts = path.split('/')
        if len(parts) > 2:
            elements.add(parts[2])
    
    print(f"Flow elements in DSS ({len(elements)}):")
    print("=" * 60)
    for elem in sorted(elements):
        print(f"  {elem}")

In [ ]:
# Plot multiple hydrographs on same figure
try:
    import matplotlib.pyplot as plt
    
    if dss_available and dss_file and dss_file.exists() and len(elements) > 0:
        fig, ax = plt.subplots(figsize=(14, 7))
        
        # Plot up to 5 elements
        for i, elem in enumerate(sorted(elements)[:5]):
            try:
                ts = HmsResults.get_outflow_timeseries(str(dss_file), elem)
                if ts is not None and not ts.empty:
                    ax.plot(ts.index, ts['flow'], linewidth=1.5, label=elem)
            except:
                continue
        
        ax.set_xlabel('Time')
        ax.set_ylabel('Flow (CFS)')
        ax.set_title('Multi-Element Hydrograph Comparison')
        ax.legend(loc='upper right')
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
except ImportError:
    print("matplotlib not installed - visualization skipped")
except Exception as e:
    print(f"Could not create multi-element plot: {e}")

## 9. Summary Statistics

In [ ]:
if dss_available and dss_file and dss_file.exists():
    print("DSS File Summary:")
    print("=" * 60)
    print(f"  File: {dss_file.name}")
    print(f"  Size: {dss_file.stat().st_size / 1024:.1f} KB")
    print(f"  Total pathnames: {len(catalog)}")
    print(f"  Flow time series: {len(flow_paths)}")
    print(f"  Precip time series: {len(precip_paths)}")
    print(f"  Flow elements: {len(elements)}")
    
    if 'peaks' in dir() and peaks is not None and not peaks.empty:
        print(f"\nPeak Flow Summary:")
        print(f"  Max peak: {peaks['peak_flow'].max():.1f} CFS")
        print(f"  Min peak: {peaks['peak_flow'].min():.1f} CFS")
        print(f"  Elements: {len(peaks)}")

## Summary

This notebook demonstrated DSS operations and results extraction:

| Class | Method | Purpose |
|-------|--------|--------|
| `HmsDss` | `is_available()` | Check if DSS operations are available |
| `HmsDss` | `get_info()` | Get DSS file metadata |
| `HmsDss` | `get_catalog()` | List all pathnames in DSS |
| `HmsDss` | `read_timeseries()` | Read individual time series |
| `HmsResults` | `get_peak_flows()` | Extract peak flows for all elements |
| `HmsResults` | `get_outflow_timeseries()` | Extract full hydrograph |

**Prerequisites**:
- Java 8+ JDK/JRE
- `pyjnius` package
- HEC Monolith libraries (auto-downloaded)

## Next Steps

- **07_execution_jython.ipynb**: Advanced execution patterns
- **08_m3_models.ipynb**: Work with HCFCD M3 models
- **05_clone_workflow.ipynb**: Compare results between scenarios